# Structure

- What the nb is about (Danilo)
- Motivating CRL (what is CRL, why we need causal representations, how do we obtain them)
- Describing Interventions-Datasets (print itnerventions and matplot some images) (Angelos)
- CITRIS-VAE (and later perhaps NF and explain NFs with some refs)
- Introduce objective and describe notation
- load pretrained citris and 
    (1) show R2 matrix, and 
    (2) go with the triplet set (i.e. fig 7) 
    (3) add noise to color (eg) and plot both images

# CITRIS: Causal Identifiability from Temporal Intervened Sequences

# Causal Representation Learning

# Temporal Intervened Sequences 

# Causal3DIdent

In [ ]:
import os
import sys
import numpy as np
import random
import matplotlib.pyplot as plt



## Imports for plotting


## PyTorch
import torch
import torch.nn as nn

# Import CITRIS-VAE
sys.path.append('CITRIS')
from pytorch_lightning import LightningModule
# from CITRIS.models.citris_vae import CITRISVAE
from CITRIS.models.citris_nf import CITRISNF

Now we will look into the Causal3DIdent dataset. It contains a sequence of images with the intervention targets at each time-step. In the below cell, we will load a small sample of the dataset.

In [4]:
causal3d_dataset = dict(np.load('causal3d.npz'))
print(causal3d_dataset.keys())

dict_keys(['images', 'interventions'])


The intervention targets delcare what causal variables were intervened. In Causal3DIdent, we have 12 causal variables: x_position, y_position, z_position, alpha_rotation, beta_rotation, gamma_rotation, theta_spot_light, hue_object, hue_spot_light, hue_background, shape and material. Below you can see what causal variables were intervened at each time-step. The i-th intervention targets imply that the causal variables of the (i-1)-th image were intervened, resulting in the i-th image.

In [ ]:
time_steps = 4
print(causal3d_dataset['interventions'][0:time_steps])

In the next cell, we will plot a sequence of images. Note how the intervention targets, printed above, affect the object and the surrounding environment.

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]
for i in range(time_steps):
    plt.subplot(1, time_steps, i+1)
    plt.imshow(causal3d_dataset['images'][i])
    plt.xticks([])
    plt.yticks([])
plt.show()

# CITRIS-VAE

# CITRIS-NF

# 

In [3]:
CHECKPOINT_PATH = "./"

pretrained_CITRIS_path = os.path.join(CHECKPOINT_PATH, "citris" + ".ckpt")


if os.path.isfile(pretrained_CITRIS_path):
    print(f"Found pretrained model at {pretrained_CITRIS_path}, loading...")
    model = CITRISNF.load_from_checkpoint(pretrained_CITRIS_path)

Found pretrained model at ./citris.ckpt, loading...


AttributeError: 'NoneType' object has no attribute 'hparams'